In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

pd.set_option("display.precision", 1)

In [3]:
def load_data(nrows=100000):

    # Load training data using pandas with a limit on the number of rows
    item_train_df = pd.read_csv('./drive/MyDrive/final_item_features.csv', nrows=nrows, dtype={'column_name': 'float32'})
    user_train_df = pd.read_csv('./drive/MyDrive/final_user_features.csv', nrows=nrows, dtype={'column_name': 'float32'})
    y_train_df = pd.read_csv('./drive/MyDrive/y_train.csv', nrows=nrows, dtype={'column_name': 'float32'})

    # Handle NaN values by replacing them with 0
    user_train_df.fillna(0, inplace=True)
    item_train_df.fillna(0, inplace=True)
    y_train_df.fillna(0, inplace=True)

    # Convert all columns except the first one to numeric
    item_train_df = item_train_df.apply(pd.to_numeric, errors='coerce')
    user_train_df = user_train_df.apply(pd.to_numeric, errors='coerce')

    return item_train_df, user_train_df, y_train_df

item_train_df, user_train_df, y_train_df = load_data()
num_user_features = user_train_df.shape[1]
num_item_features = item_train_df.shape[1]

# Handle NaN values by replacing them with 0 (again)
user_train_df.fillna(0, inplace=True)
item_train_df.fillna(0, inplace=True)
y_train_df.fillna(0, inplace=True)

# Print out shapes and samples to inspect the data
print(f"User Training Data Shape: {user_train_df.shape}")
print(f"Item Training Data Shape: {item_train_df.shape}")
print(f"y_train Shape: {y_train_df.shape}")

# Display sample data
print("Sample User Features (first 5):")
print(user_train_df.head())
print("Sample Item Features (first 5):")
print(item_train_df.head())
print("Sample Ratings (first 5):")
print(y_train_df.head())

<ipython-input-3-633dda195d32>:4: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  item_train_df = pd.read_csv('./drive/MyDrive/final_item_features.csv', nrows=nrows, dtype={'column_name': 'float32'})


User Training Data Shape: (100000, 31)
Item Training Data Shape: (100000, 35)
y_train Shape: (100000, 3)
Sample User Features (first 5):
   rating_count  rating_average  Mexican  Chinese  Italian  Irish  Pizza  \
0             3             2.7      0.0      0.0      0.0    0.0    4.0   
1             3             2.7      0.0      0.0      0.0    0.0    4.0   
2             3             2.7      0.0      0.0      0.0    0.0    4.0   
3            31             4.3      4.2      4.0      4.5    3.0    4.0   
4            31             4.3      4.2      4.0      4.5    3.0    4.0   

   Bars  Nightlife  Japanese  ...  Ambience_classy  Ambience_trendy  \
0   2.5        2.5       0.0  ...              4.0              0.0   
1   2.5        2.5       0.0  ...              4.0              0.0   
2   2.5        2.5       0.0  ...              4.0              0.0   
3   4.1        4.2       0.0  ...              4.8              4.7   
4   4.1        4.2       0.0  ...              4.8 

In [4]:
def prepare_training_data(user_train_df, item_train_df, y_train_df):

    # Convert DataFrames to NumPy arrays
    user_train = user_train_df.to_numpy()
    item_train = item_train_df.to_numpy()
    y_train = y_train_df.iloc[:, 2].to_numpy()

    # Reshape y_train to be a column vector
    y_train = y_train.reshape(-1, 1)

    return user_train, item_train, y_train

user_train, item_train, y_train = prepare_training_data(user_train_df, item_train_df, y_train_df)

# Print out shapes of the prepared training data
print("Converted User Training Data Shape:", user_train.shape)
print("Converted Item Training Data Shape:", item_train.shape)
print("Converted Ratings Shape:", y_train.shape)
print("Converted Item Training Data Shape:", item_train)

Converted User Training Data Shape: (100000, 31)
Converted Item Training Data Shape: (100000, 35)
Converted Ratings Shape: (100000, 1)
Converted Item Training Data Shape: [[  4.  164.    2.  ...   0.    1.    0. ]
 [  3.  347.    2.  ...   0.    1.    0. ]
 [  4.5 114.    1.  ...   0.    0.    0. ]
 ...
 [  4.5 139.    2.  ...   0.    0.    0. ]
 [  4.  171.    1.  ...   0.    0.    0. ]
 [  4.5  43.    2.  ...   0.    0.    0. ]]


In [5]:
# Ensure all data is in float format
item_train = item_train.astype(np.float32)
user_train = user_train.astype(np.float32)
y_train = y_train.astype(np.float32)

# Scale data
item_train_unscaled = item_train.copy()
user_train_unscaled = user_train.copy()
y_train_unscaled = y_train.copy()

scalerItem = StandardScaler()
scalerUser = StandardScaler()
scalerTarget = MinMaxScaler((-1, 1))

item_train = scalerItem.fit_transform(item_train)
user_train = scalerUser.fit_transform(user_train)
y_train = scalerTarget.fit_transform(y_train.reshape(-1, 1)).flatten()

In [6]:
# Train-Test Split
user_train, user_test = train_test_split(user_train, train_size=0.7, shuffle=True, random_state=1)
item_train, item_test = train_test_split(item_train, train_size=0.7, shuffle=True, random_state=1)
y_train, y_test = train_test_split(y_train, train_size=0.7, shuffle=True, random_state=1)

print(f"User training data shape: {user_train.shape}")
print(f"Item (Restaurant) training data shape: {item_train.shape}")

User training data shape: (70000, 31)
Item (Restaurant) training data shape: (70000, 35)


In [7]:
# Define Neural Network Models
class UserNN(nn.Module):
    def __init__(self, num_user_features, num_outputs):
        super(UserNN, self).__init__()
        self.fc1 = nn.Linear(num_user_features, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, num_outputs)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

class ItemNN(nn.Module):
    def __init__(self, num_item_features, num_outputs):
        super(ItemNN, self).__init__()
        self.fc1 = nn.Linear(num_item_features, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, num_outputs)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

In [8]:
device = torch.device("cuda")

# Instantiate models with half precision
num_outputs = 64  # Embedding size
user_nn = UserNN(num_user_features, num_outputs).to(device).half()
item_nn = ItemNN(num_item_features, num_outputs).to(device).half()

# Create inputs in half precision and move them to GPU
input_user = torch.tensor(user_train, dtype=torch.float16).to(device)
input_item = torch.tensor(item_train, dtype=torch.float16).to(device)

# Forward pass through user and item networks
vu = user_nn(input_user)
vu = nn.functional.normalize(vu, dim=1)

vm = item_nn(input_item)
vm = nn.functional.normalize(vm, dim=1)

# Define a smaller batch size to avoid memory issues
batch_size = 64

# Keep the output tensor on CPU to save GPU memory
output = torch.zeros(input_user.shape[0], item_train.shape[0], dtype=torch.float16)

# Mini-batch processing
for start in range(0, input_user.shape[0], batch_size):
    end = start + batch_size
    user_batch = vu[start:end].to(device)

    # Compute the scores for this batch in float16
    batch_output = torch.mm(user_batch, vm.t())

    # Move the batch output back to CPU and store in the output tensor
    output[start:end] = batch_output.cpu()

# Now `output` contains the predicted scores for all users and items on the CPU in float16

In [9]:
# Combine Models into a Recommendation Model
class RecommendationModel(nn.Module):
    def __init__(self, user_nn, item_nn):
        super(RecommendationModel, self).__init__()
        self.user_nn = user_nn
        self.item_nn = item_nn

    def forward(self, user_input, item_input):
        vu = self.user_nn(user_input)
        vu = nn.functional.normalize(vu, dim=1)
        vm = self.item_nn(item_input)
        vm = nn.functional.normalize(vm, dim=1)
        output = torch.bmm(vu.unsqueeze(1), vm.unsqueeze(2)).squeeze()
        return output

model = RecommendationModel(user_nn, item_nn)
print(model)

RecommendationModel(
  (user_nn): UserNN(
    (fc1): Linear(in_features=31, out_features=256, bias=True)
    (fc2): Linear(in_features=256, out_features=128, bias=True)
    (fc3): Linear(in_features=128, out_features=64, bias=True)
  )
  (item_nn): ItemNN(
    (fc1): Linear(in_features=35, out_features=256, bias=True)
    (fc2): Linear(in_features=256, out_features=128, bias=True)
    (fc3): Linear(in_features=128, out_features=64, bias=True)
  )
)


In [10]:
# Set random seed for reproducibility
torch.manual_seed(1)

# Define loss function and optimizer
cost_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [11]:
# Train the Model
num_epochs = 170

model = model.to(device).float()

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Prepare data and move to the same device as the model
    user_batch = torch.tensor(user_train, dtype=torch.float32).to(device)
    item_batch = torch.tensor(item_train, dtype=torch.float32).to(device)
    y_batch = torch.tensor(y_train, dtype=torch.float32).squeeze().to(device)

    # Forward pass
    predictions = model(user_batch, item_batch)

    # Compute loss
    loss = cost_fn(predictions, y_batch)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 5 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [5/170], Loss: 0.3151
Epoch [10/170], Loss: 0.3028
Epoch [15/170], Loss: 0.3070
Epoch [20/170], Loss: 0.2859
Epoch [25/170], Loss: 0.2765
Epoch [30/170], Loss: 0.2716
Epoch [35/170], Loss: 0.2623
Epoch [40/170], Loss: 0.2508
Epoch [45/170], Loss: 0.2424
Epoch [50/170], Loss: 0.2373
Epoch [55/170], Loss: 0.2347
Epoch [60/170], Loss: 0.2323
Epoch [65/170], Loss: 0.2303
Epoch [70/170], Loss: 0.2289
Epoch [75/170], Loss: 0.2276
Epoch [80/170], Loss: 0.2266
Epoch [85/170], Loss: 0.2256
Epoch [90/170], Loss: 0.2248
Epoch [95/170], Loss: 0.2241
Epoch [100/170], Loss: 0.2234
Epoch [105/170], Loss: 0.2228
Epoch [110/170], Loss: 0.2223
Epoch [115/170], Loss: 0.2217
Epoch [120/170], Loss: 0.2212
Epoch [125/170], Loss: 0.2207
Epoch [130/170], Loss: 0.2202
Epoch [135/170], Loss: 0.2198
Epoch [140/170], Loss: 0.2193
Epoch [145/170], Loss: 0.2188
Epoch [150/170], Loss: 0.2183
Epoch [155/170], Loss: 0.2177
Epoch [160/170], Loss: 0.2172
Epoch [165/170], Loss: 0.2167
Epoch [170/170], Loss: 0.2161


In [12]:
# Evaluate the Model
model.eval()
with torch.no_grad():
    user_test_tensor = torch.tensor(user_test, dtype=torch.float32).to(device)
    item_test_tensor = torch.tensor(item_test, dtype=torch.float32).to(device)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32).squeeze().to(device)

    # Make predictions
    test_predictions = model(user_test_tensor, item_test_tensor)

    # Compute test loss
    test_loss = cost_fn(test_predictions, y_test_tensor)
    print(f'Test Loss: {test_loss.item():.4f}')

Test Loss: 0.2283


In [13]:
# Save the Model Weights
model_path = 'model_weights.pth'
torch.save(model.state_dict(), model_path)
print(f'Model weights saved to {model_path}')

Model weights saved to model_weights.pth


In [14]:
# Example: New user
new_user = np.array([[3, 4.0, 4.0, 0.0, 0.0, 0.0, 0.0, 4.0, 4.0, 0.0, 0.0, 4.0, 0.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 4.0, 0, 0, 0, 2.0, 0.6666666666666666, 0.6666666666666666]])  # Modify based on relevant features
new_user_scaled = scalerUser.transform(new_user)

# Convert to tensor and predict
new_user_tensor = torch.tensor(new_user_scaled, dtype=torch.float32).to(device)

# Get all item features and convert to tensor
item_tensor = torch.tensor(item_train, dtype=torch.float32).to(device)

# Repeat new user tensor for all items to create the correct input shape
new_user_batch = new_user_tensor.repeat(item_tensor.shape[0], 1)

# Make predictions for all items
predictions = model(new_user_batch, item_tensor)

# Inverse scale and show top 10 recommendations
predictions_inverse = scalerTarget.inverse_transform(predictions.reshape(-1, 1).detach().cpu().numpy())
top_10_indices = predictions_inverse.flatten().argsort()[-10:][::-1]

print("Top 10 Restaurant Recommendations for New User:")
print(item_train_df.iloc[top_10_indices])


Top 10 Restaurant Recommendations for New User:
       stars  review_count  RestaurantsPriceRange2  WiFi  alcohol  NoiseLevel  \
32397    4.5           230                     2.0     0        0         1.0   
30846    4.5           183                     2.0     1        1         2.0   
24693    3.5            51                     3.0     0        2         2.0   
25008    4.0           161                     2.0     1        2         2.0   
455      4.0           371                     1.0     1        0         2.0   
32371    4.0            30                     1.0     0        0         1.0   
16962    4.5            25                     1.0     0        0         2.0   
65533    2.0            36                     1.0     0        0         2.0   
52470    3.5           165                     3.0     1        2         2.0   
14510    3.5             6                     1.0     0        0         0.0   

       RestaurantsGoodForGroups  GoodForKids  HasTV  attire 